<div style="background:#E9FFF6; color:#440404; padding:8px; border-radius: 4px; text-align: center; font-weight: 500;">IFQ619 - Data Analytics for Strategic Decision Makers (2024)</div>

# IFQ619 :: C1-UnstructuredAnalytics

For this session, the focus will be on analysis of unstructured text. However, the thinking required is similar to approaches to analysing images, video, sound and other unstructured data. Primarily, the analysis is based on the notion that there are useful patterns in the unstructured data which can be obtained mathematically. By converting the data to a mathematical structure, various algorithms can be applied to the structure with the aim of identifying patterns. 

In the case of the `topic modelling` approaches below, many of the techniques are *probabilistic* - that is they mathematically identify the *likelihood* that a feature might be important. Thus, they are never 100% accurate, and their use needs to be mediated by a more pragmatic *useful or not* approach, rather than *right or wrong*.

In [1]:
# Import the necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation, NMF
import pandas as pd
import json
import random
from sklearn.cluster import KMeans

### Accessing the data via The Guardian API

See the `Accessing_the_Guardian_API.ipynb` notebook file for details on getting the data. **Note:** This approach may be used for additional data for Assignment 2.

### Read in pre-saved data

To save time, we're loading in pre-saved data that was fetched using the Guardian API.

In [2]:
# Load the data - articles from The Guardian about the war in Ukraine
file_path = "data/"
file_name = "qld_rural_articles.json"

with open(f"{file_path}{file_name}",'r', encoding='utf-8') as fp:
    articles = json.load(fp)

print(f"Loaded {len(articles)} articles from {file_name}")

Loaded 318 articles from qld_rural_articles.json


Each dictionary entry includes the *title [date]* as `key` and the *body text* from the article as `value`.

In [3]:
article1 = list(articles.items())[0]
print("Key:",article1[0])
print("Value:",article1[1][:300],"...") # Just show first 300 characters

Key: Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z]
Value: Queensland has been declared completely free of drought for the first time in 11 years. Months of flooding rains have transformed much of the state’s dusty red-earth interior into a network of swollen creeks and rivers awash with green shoots and birdlife. Diamantina and Bulloo shires, located more  ...


So the values gives us a list of documents that we can analyse.

In [4]:
# Get a list of documents
documents = list(articles.values())

# View first 400 characters of the 1st document
documents[0][:400]

'Queensland has been declared completely free of drought for the first time in 11 years. Months of flooding rains have transformed much of the state’s dusty red-earth interior into a network of swollen creeks and rivers awash with green shoots and birdlife. Diamantina and Bulloo shires, located more than 1000km west of Brisbane in Queensland’s channel country, this month became the last two local g'

### Term Count 

**Finding important terms by the frequency of their occurance**

Using `CountVectorizer` create a `vector` for each document where the dimensionality of the vector is the `vocabulary` (all terms in the collection), and the value of each component is the number of times that the `term` occurs in the document.

All of these analyses, approach the document as a [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. In this approach, the order of the words don't matter. A popular approach that takes into account order is [Word embedding](https://en.wikipedia.org/wiki/Word_embedding). This session does not explore word embedding.

In [5]:
# AYOOOOOO WE DOING A LEMMATISING FUNCTION
# https://jonathansoma.com/lede/image-and-sound/text-analysis/text-analysis-word-counting-lemmatizing-and-tf-idf/

import spacy
nlp = spacy.load("en_core_web_sm")
# Source: spaCy is a dream, but a dream where sometimes your legs won’t move right and you can’t read text. But sometimes you can fly! So yes, as always, ups and downs
# Morgan: I hate this
# python -m spacy download en
# python -m pip install spacy

def lemmatize(text):
    doc = nlp(text)
    # Turn it into tokens, ignoring the punctuation
    tokens = [token for token in doc if not token.is_punct]
    # Convert those tokens into lemmas, EXCEPT the pronouns, we'll keep those.
    lemmas = [token.lemma_ if token.pos_ != 'PRON' else token.orth_ for token in tokens]
    return lemmas

In [6]:
# Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# Count a maximum of 10000 terms, and remove common english stop words
StopWords = list(ENGLISH_STOP_WORDS.union(["budget", "funding", "Monday","Tuesday","Wednesday","Thursday","Friday", "Saturday", "Sunday","nbsp", "\n", "|", "\n "]))
import re

#count_vectorizer = CountVectorizer(preprocessor=lambda x: x.replace('(\D+)', "", regex = True))
#count_vectorizer = CountVectorizer(preprocessor=lambda x: re.sub(r"([\d\.])+", '"NUM'", x.lower()),
count_vectorizer = CountVectorizer(
                                   max_df=0.75,min_df=2,max_features=10000,stop_words=StopWords, tokenizer=lemmatize)
count_dt_matrix = count_vectorizer.fit_transform(articles.values())



c:\Users\Morgan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Morgan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'friday', 'make', 'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday', 'whereaft'] not in stop_words.
  warnings.warn(


**I HAVE ADDED MY OWN ADDITIONAL STOP WORDS HERE FOR NUMBERS**
NOt sufficient, referred to https://stackoverflow.com/questions/43216530/adding-numbers-to-stop-words-to-scikit-learns-countvectorizer

In [7]:
# Take a look at the vector for the first document
doc001_vector = count_dt_matrix.toarray()[0]
doc001_vector

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [8]:
# Get the 1000 terms identified during the vectorization process
feature_names = count_vectorizer.get_feature_names_out()
feature_names

array(['\n \n', '\n \n ', '\n \n  ', ..., '’re', '’ve', '“it'],
      dtype=object)

In [9]:
# Look at how the counts match up to the terms (for the 1st doc)
#doc001_term_counts = list(zip(feature_names,doc001_vector[0])) 
#doc001_term_counts

In [10]:
# Take a look at the vocabulary which shows the total counts for whole collection
count_vectorizer.vocabulary_

{'declare': 2019,
 'completely': 1619,
 'free': 2926,
 'drought': 2331,
 'time': 6692,
 '11': 64,
 'month': 4410,
 'flooding': 2843,
 'rain': 5373,
 'transform': 6774,
 'state': 6285,
 'dusty': 2353,
 'red': 5474,
 'earth': 2366,
 'interior': 3641,
 'network': 4544,
 'swollen': 6518,
 'creek': 1864,
 'river': 5723,
 'green': 3139,
 'shoot': 6012,
 'birdlife': 1030,
 'diamantina': 2147,
 'shire': 6003,
 'locate': 4054,
 '1000': 60,
 'km': 3830,
 'west': 7193,
 'brisbane': 1164,
 'channel': 1394,
 'country': 1825,
 'local': 4050,
 'government': 3106,
 'area': 754,
 'declaration': 2018,
 'officially': 4670,
 'revoke': 5677,
 '2013': 181,
 ' ': 12,
 'receive': 5442,
 'fortnightly': 2901,
 'newsletter': 4557,
 'burn': 1215,
 'jo': 3731,
 'sheppard': 5996,
 'chief': 1428,
 'executive': 2618,
 'farmer': 2723,
 'federation': 2754,
 'experience': 2642,
 'grow': 3167,
 'cattle': 1342,
 'station': 6290,
 'neighbouring': 4535,
 '10': 56,
 'incredibly': 3512,
 'difficult': 2160,
 'endure': 2489,
 '

In [11]:
count_vectorizer.vocabulary_["regional"]

5509

#### Display matrix in dataframe

Take the term count matrix and display in a dataframe to make visible the structure


In [12]:
count_dt_matrix.toarray()

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 2, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
# Create a new dataframe with the matrix - use titles for the index and terms for the columns
count_df = pd.DataFrame(count_dt_matrix.toarray(), index=articles.keys(), columns=feature_names)
count_df

,\n \n,\n \n,\n \n,\n \n \n \n,\n \n \n,\n \n \n,\n \n \n \n,\n \n \n,\n \n \n \n \n \n \n,\n \n \n \n \n \n,...,zero,zoe,zone,zoo,’d,’ll,’m,’re,’ve,“it
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,0
Queensland rejects Glencore carbon capture and storage proposal for Great Artesian Basin [2024-05-24T06:12:49Z],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Antarctic blast brings damaging winds and alpine blizzards to Victoria and NSW and possible snow in Queensland [2024-07-15T06:06:30Z],1,2,1,0,0,0,0,0,1,0,...,0,0,0,0,1,1,1,0,0,0
"300,000ha Queensland cattle station bought for conservation after $21m donation [2024-04-16T23:00:03Z]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Contrasting polls spell out disaster territory and green shoots for Labor ahead of Queensland election [2024-09-14T20:00:39Z],1,2,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Coalition won’t budge on free rapid Covid tests for all but concession card-holders are covered [2022-01-05T09:13:17Z],0,1,3,1,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
"Over-12s should be able to get Covid vaccine in Australia without parental consent, say experts [2022-01-24T03:48:47Z]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
What do I do if I test positive for Covid? Here’s how to prepare for the result [2022-01-04T00:46:17Z],0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,0
NSW government to create a new reconstruction authority in response to flood report [2022-08-16T17:30:13Z],0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


By selecting a row from the dataframe and sorting the values (counts), we can identify the top 10 terms

#### Create a top10 terms dataframe

Using the index from the documents, create a dataframe that can hold the top10 terms for each document. We also include columns for our other analysis (tfidf, lda, nmf)

In [14]:
# Create a dataframe to hold top terms for each analysis type
terms_df = pd.DataFrame(index=count_df.index,columns=['count','tfidf','lda','nmf'])
terms_df

,count,tfidf,lda,nmf
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],NaN,NaN,NaN,NaN
Queensland rejects Glencore carbon capture and storage proposal for Great Artesian Basin [2024-05-24T06:12:49Z],NaN,NaN,NaN,NaN
Antarctic blast brings damaging winds and alpine blizzards to Victoria and NSW and possible snow in Queensland [2024-07-15T06:06:30Z],NaN,NaN,NaN,NaN
"300,000ha Queensland cattle station bought for conservation after $21m donation [2024-04-16T23:00:03Z]",NaN,NaN,NaN,NaN
Contrasting polls spell out disaster territory and green shoots for Labor ahead of Queensland election [2024-09-14T20:00:39Z],NaN,NaN,NaN,NaN
...,...,...,...,...
Coalition won’t budge on free rapid Covid tests for all but concession card-holders are covered [2022-01-05T09:13:17Z],NaN,NaN,NaN,NaN
"Over-12s should be able to get Covid vaccine in Australia without parental consent, say experts [2022-01-24T03:48:47Z]",NaN,NaN,NaN,NaN
What do I do if I test positive for Covid? Here’s how to prepare for the result [2022-01-04T00:46:17Z],NaN,NaN,NaN,NaN
NSW government to create a new reconstruction authority in response to flood report [2022-08-16T17:30:13Z],NaN,NaN,NaN,NaN


Populate the count column with data created by the count vectorizer.

In [15]:
#For each doc, get the 10 columns with the largest counts
for idx in terms_df.index:
    counts = dict(count_df.loc[idx].sort_values(ascending=False).head(10))
    #print(counts)
    terms_df.at[idx,'count'] = list(counts.keys()) # Just the list of terms

terms_df

,count,tfidf,lda,nmf
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],"[drought, rain, country, state, network, farme...",NaN,NaN,NaN
Queensland rejects Glencore carbon capture and storage proposal for Great Artesian Basin [2024-05-24T06:12:49Z],"[project, ccs, basin, artesian, great, glencor...",NaN,NaN,NaN
Antarctic blast brings damaging winds and alpine blizzards to Victoria and NSW and possible snow in Queensland [2024-07-15T06:06:30Z],"[snow, south, weather, bring, condition, wind,...",NaN,NaN,NaN
"300,000ha Queensland cattle station bought for conservation after $21m donation [2024-04-16T23:00:03Z]","[government, protect, national, park, area, pr...",NaN,NaN,NaN
Contrasting polls spell out disaster territory and green shoots for Labor ahead of Queensland election [2024-09-14T20:00:39Z],"[labor, party, seat, state, brisbane, election...",NaN,NaN,NaN
...,...,...,...,...
Coalition won’t budge on free rapid Covid tests for all but concession card-holders are covered [2022-01-05T09:13:17Z],"[test, free, rapid, morrison, testing, governm...",NaN,NaN,NaN
"Over-12s should be able to get Covid vaccine in Australia without parental consent, say experts [2022-01-24T03:48:47Z]","[child, parent, consent, medical, vaccinate, v...",NaN,NaN,NaN
What do I do if I test positive for Covid? Here’s how to prepare for the result [2022-01-04T00:46:17Z],"[covid, health, positive, home, isolation, iso...",NaN,NaN,NaN
NSW government to create a new reconstruction authority in response to flood report [2022-08-16T17:30:13Z],"[flood, agency, state, nsw, government, perrot...",NaN,NaN,NaN


In [16]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = count_df.iloc[sample]
    top_terms = dict(count_df.iloc[sample].sort_values(ascending=False).head(10))
    print(f"[{sample}] {doc.name}")
    print("\t- Top terms:",top_terms)

[52] How many cattle are there in Australia? We may be out by 10 million [2024-03-10T14:00:05Z]
	- Top terms: {'cattle': 20, 'datum': 11, 'million': 8, 'industry': 8, 'meat': 7, 'emission': 7, 'estimate': 7, 'fordyce': 7, 'walter': 6, 'number': 6}
[28] After 25 years, logging and bushfires, a greater glider has been spotted in Deongwar state forest [2024-04-30T15:00:33Z]
	- Top terms: {'forest': 17, 'state': 12, 'great': 9, 'glider': 9, 'tree': 9, 'hectare': 6, 'area': 6, 'habitat': 5, 'walsh': 5, ' ': 4}
[4] Contrasting polls spell out disaster territory and green shoots for Labor ahead of Queensland election [2024-09-14T20:00:39Z]
	- Top terms: {'labor': 13, 'party': 9, 'seat': 8, 'state': 6, 'brisbane': 6, 'election': 6, 'green': 4, 'poll': 4, 'redbridge': 4, 'term': 4}
[234] Archie Moore becomes second solo Aboriginal artist to represent Australia at Venice Biennale [2023-02-08T01:10:21Z]
	- Top terms: {'artist': 9, 'moore': 9, 'represent': 8, 'work': 8, 'nation': 6, 'art': 6, 'mem

### Term Frequency / Inverse Document Frequency (TF/IDF)

**Finding terms that are very common in a document, but less common in the whole collection**

The [TF/IDF](https://en.wikipedia.org/wiki/Tf–idf) algorithm takes the term frequencies for a document and divides them by the frequencies of the terms in the whole collection.


In [17]:
# Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# Count a maximum of 10000 terms, and remove common english stop words
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.75, min_df=2, max_features=10000, stop_words="english"
)

In [18]:
# Get the document vectors
tfidf_dt_matrix = tfidf_vectorizer.fit_transform(articles.values())

# Display the vector for the first document
tfidf_dt_matrix.toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

#### Display matrix in dataframe

In [19]:
tfidf_df = pd.DataFrame(tfidf_dt_matrix.toarray(), index=articles.keys(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

,000,000ha,000km,027,069,0808,0c,10,100,1000,...,yvette,yvonne,zali,zealand,zelenskiy,zero,zoe,zone,zones,zoo
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.022430,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
Queensland rejects Glencore carbon capture and storage proposal for Great Artesian Basin [2024-05-24T06:12:49Z],0.016627,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
Antarctic blast brings damaging winds and alpine blizzards to Victoria and NSW and possible snow in Queensland [2024-07-15T06:06:30Z],0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
"300,000ha Queensland cattle station bought for conservation after $21m donation [2024-04-16T23:00:03Z]",0.000000,0.230648,0.0,0.0,0.0,0.0,0.0,0.022531,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
Contrasting polls spell out disaster territory and green shoots for Labor ahead of Queensland election [2024-09-14T20:00:39Z],0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Coalition won’t budge on free rapid Covid tests for all but concession card-holders are covered [2022-01-05T09:13:17Z],0.031004,0.000000,0.0,0.0,0.0,0.0,0.0,0.023450,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
"Over-12s should be able to get Covid vaccine in Australia without parental consent, say experts [2022-01-24T03:48:47Z]",0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
What do I do if I test positive for Covid? Here’s how to prepare for the result [2022-01-04T00:46:17Z],0.051824,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.021342,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.021342,0.0,0.0,0.0,0.0
NSW government to create a new reconstruction authority in response to flood report [2022-08-16T17:30:13Z],0.021115,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.064587,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


#### Update the terms matrix

In [20]:
for idx in terms_df.index:
    tfidf = dict(tfidf_df.loc[idx].sort_values(ascending=False).head(10))
    #print(counts)
    terms_df.at[idx,'tfidf'] = list(tfidf.keys()) 

terms_df

,count,tfidf,lda,nmf
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],"[drought, rain, country, state, network, farme...","[drought, rains, sheppard, channel, monaghan, ...",NaN,NaN
Queensland rejects Glencore carbon capture and storage proposal for Great Artesian Basin [2024-05-24T06:12:49Z],"[project, ccs, basin, artesian, great, glencor...","[ccs, artesian, glencore, basin, project, co2,...",NaN,NaN
Antarctic blast brings damaging winds and alpine blizzards to Victoria and NSW and possible snow in Queensland [2024-07-15T06:06:30Z],"[snow, south, weather, bring, condition, wind,...","[snow, belt, altitude, weather, metres, winds,...",NaN,NaN
"300,000ha Queensland cattle station bought for conservation after $21m donation [2024-04-16T23:00:03Z]","[government, protect, national, park, area, pr...","[000ha, park, linard, nature, opal, property, ...",NaN,NaN
Contrasting polls spell out disaster territory and green shoots for Labor ahead of Queensland election [2024-09-14T20:00:39Z],"[labor, party, seat, state, brisbane, election...","[labor, seats, redbridge, party, election, bri...",NaN,NaN
...,...,...,...,...
Coalition won’t budge on free rapid Covid tests for all but concession card-holders are covered [2022-01-05T09:13:17Z],"[test, free, rapid, morrison, testing, governm...","[tests, nbsp, rapid, testing, morrison, antige...",NaN,NaN
"Over-12s should be able to get Covid vaccine in Australia without parental consent, say experts [2022-01-24T03:48:47Z]","[child, parent, consent, medical, vaccinate, v...","[consent, vaccine, parents, mature, medical, p...",NaN,NaN
What do I do if I test positive for Covid? Here’s how to prepare for the result [2022-01-04T00:46:17Z],"[covid, health, positive, home, isolation, iso...","[covid, isolation, isolate, positive, health, ...",NaN,NaN
NSW government to create a new reconstruction authority in response to flood report [2022-08-16T17:30:13Z],"[flood, agency, state, nsw, government, perrot...","[perrottet, agency, flood, resilience, disaste...",NaN,NaN


#### Compare approaches

In [21]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print()

[79] Rupert Murdoch’s son-in-law and the soil carbon revolution  [2024-03-01T23:00:14Z]
	- Counts:	 ['carbon', 'soil', 'credit', 'wilmot', 'farmer', 'emission', 'project', 'sequestration', 'long', 'sell']
	- TFIDF:	 ['carbon', 'soil', 'wilmot', 'credits', 'says', 'sequestration', 'emissions', 'farmers', 'science', 'product']

[208] After Aston, Liberal moderates can hear the bell toll. But is Peter Dutton listening? | Paul Karp [2023-04-07T15:00:08Z]
	- Counts:	 ['party', 'liberal', 'aston', 'loss', 'election', 'government', 'albanese', 'city', ' ', 'need']
	- TFIDF:	 ['aston', 'party', 'liberal', 'liberals', 'archer', 'loss', 'albanese', 'gallery', 'dutton', 'election']

[135] Australia’s Kangaroo Island named No 2 must-visit region for 2024 by Lonely Planet [2023-10-25T14:00:07Z]
	- Counts:	 ['island', 'kangaroo', 'good', 'beach', 'planet', 'lonely', 'pick', '2024', 'just', 'come']
	- TFIDF:	 ['island', 'kangaroo', 'lonely', 'planet', 'mainland', 'best', 'beaches', 'beach', 'picked',

### Topic modelling with Latent Dirichlet Allocation (LDA)

[LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is an algorithm for obtaining *topics* (a list of terms) from a document-term matrix. It is a generative probabilistic approach to *decomposition* of the document-term matrix into 2 factor matrices: document-topic and topic-term.

![img](https://editor.analyticsvidhya.com/uploads/26864dtm.JPG)

*Source: [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/06/part-2-topic-modeling-and-latent-dirichlet-allocation-lda-using-gensim-and-sklearn/)*

The LDA model requires the number of topics to be set in advance. As it is a generative model, it also runs over a number of iterations. These values usually need to be experimented with to obtain quality topics.

In [22]:
# Set number of topics
num_topics = 20
# Set max number of iteractions
max_iterations = 50

# Create the model
lda_model = LatentDirichletAllocation(n_components=num_topics,max_iter=max_iterations,learning_method='online')

# Fit the model to the data, and use the model to transform the data (do the decomposition)
doc_topic_matrix = lda_model.fit_transform(count_dt_matrix) # Note that this uses the counts already processed

# Obtain the topics
topic_term_matrix = lda_model.components_

#### View the topics

In [23]:
# Get the topics and their terms
lda_topic_dict = {}
for index, topic in enumerate(topic_term_matrix):
    zipped = zip(feature_names, topic)
    top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
    #print(top_terms)
    top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
    lda_topic_dict[f"topic_{index}"] = top_terms_list

# Print the topics with their terms    
for k,v in lda_topic_dict.items():
    print(k)
    print(v)
    print()

topic_0
{'farmer': 0.0506, 'cattle': 0.0506, 'daylight': 0.0506, 'drought': 0.0506, 'saving': 0.0505, 'carbon': 0.0504, 'school': 0.0504, 'beef': 0.0504, 'state': 0.0504, 'time': 0.0504}

topic_1
{'bonza': 28.7076, 'airline': 24.8245, 'airport': 16.9458, 'passenger': 12.9127, 'aircraft': 12.6387, 'aviation': 10.6474, 'transport': 9.5883, 'king': 9.5391, 'flight': 8.6671, 'qanta': 7.9285}

topic_2
{'news': 123.4134, 'australian': 98.2433, 'medium': 98.2228, 'world': 94.2418, 'morning': 83.9687, 'police': 83.5035, 'day': 72.5873, 'story': 70.3863, 'woman': 61.7125, 'sydney': 59.7582}

topic_3
{'adani': 17.8298, 'carmichael': 7.3318, 'advantaged': 6.4795, 'rank': 4.4914, 'coordinator': 4.0254, 'legal': 4.0074, 'qld': 3.9601, 'disadvantaged': 3.9425, 'fly': 3.4579, 'employment': 2.9695}

topic_4
{'tree': 30.3421, 'pesticide': 18.6196, 'horse': 17.7355, 'result': 16.8749, 'fish': 15.2258, 'testing': 13.703, 'sample': 13.1415, 'epa': 12.3002, 'level': 11.8439, 'water': 11.4887}

topic_5
{'ga

#### List of topics for each document

In [24]:
doc_topic_matrix[0]

array([1.59235669e-04, 1.59235669e-04, 1.59235671e-04, 1.59235670e-04,
       1.59235672e-04, 1.59235676e-04, 1.59235672e-04, 1.59235669e-04,
       1.59235670e-04, 1.59235672e-04, 6.52866256e-03, 1.59235670e-04,
       1.59235673e-04, 1.59235669e-04, 1.21365577e-01, 8.69398754e-01,
       1.59235669e-04, 1.59235671e-04, 1.59235672e-04, 1.59235669e-04])

In [25]:
#doc_topic_matrix[200]

#### Update the terms matrix

In [26]:
for idx,topic in enumerate(doc_topic_matrix):
     # enumerate just takes a list and returns the value and a counter for the loop starting from 0
    topic_num = topic.argmax()
    top_topic = lda_topic_dict[f"topic_{topic_num}"]
    terms_df['lda'].iloc[idx] = list(top_topic.keys())

terms_df

C:\Users\Morgan\AppData\Local\Temp\ipykernel_18348\474454722.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  terms_df['lda'].iloc[idx] = list(top_topic.keys())


,count,tfidf,lda,nmf
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],"[drought, rain, country, state, network, farme...","[drought, rains, sheppard, channel, monaghan, ...","[government, people, state, make, , community...",NaN
Queensland rejects Glencore carbon capture and storage proposal for Great Artesian Basin [2024-05-24T06:12:49Z],"[project, ccs, basin, artesian, great, glencor...","[ccs, artesian, glencore, basin, project, co2,...","[government, people, state, make, , community...",NaN
Antarctic blast brings damaging winds and alpine blizzards to Victoria and NSW and possible snow in Queensland [2024-07-15T06:06:30Z],"[snow, south, weather, bring, condition, wind,...","[snow, belt, altitude, weather, metres, winds,...","[government, people, state, make, , community...",NaN
"300,000ha Queensland cattle station bought for conservation after $21m donation [2024-04-16T23:00:03Z]","[government, protect, national, park, area, pr...","[000ha, park, linard, nature, opal, property, ...","[government, people, state, make, , community...",NaN
Contrasting polls spell out disaster territory and green shoots for Labor ahead of Queensland election [2024-09-14T20:00:39Z],"[labor, party, seat, state, brisbane, election...","[labor, seats, redbridge, party, election, bri...","[government, people, state, make, , community...",NaN
...,...,...,...,...
Coalition won’t budge on free rapid Covid tests for all but concession card-holders are covered [2022-01-05T09:13:17Z],"[test, free, rapid, morrison, testing, governm...","[tests, nbsp, rapid, testing, morrison, antige...","[government, people, state, make, , community...",NaN
"Over-12s should be able to get Covid vaccine in Australia without parental consent, say experts [2022-01-24T03:48:47Z]","[child, parent, consent, medical, vaccinate, v...","[consent, vaccine, parents, mature, medical, p...","[government, people, state, make, , community...",NaN
What do I do if I test positive for Covid? Here’s how to prepare for the result [2022-01-04T00:46:17Z],"[covid, health, positive, home, isolation, iso...","[covid, isolation, isolate, positive, health, ...","[government, people, state, make, , community...",NaN
NSW government to create a new reconstruction authority in response to flood report [2022-08-16T17:30:13Z],"[flood, agency, state, nsw, government, perrot...","[perrottet, agency, flood, resilience, disaste...","[government, people, state, make, , community...",NaN


#### Compare approaches

In [27]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print("\t- LDA:\t\t",doc['lda'])
    print()

[43] In rural Australia, everyone knows your business – so it's harder for victims of violence to get help | Gabrielle Chan [2024-05-06T15:00:42Z]
	- Counts:	 ['woman', 'partner', 'mclachlan', 'violence', 'area', 'know', 'great', 'male', 'gun', 'town']
	- TFIDF:	 ['mclachlan', 'violence', 'partner', 'women', 'intimate', 'male', 'greater', 'guns', 'areas', 'victim']
	- LDA:		 ['government', 'people', 'state', 'make', ' ', 'community', 'need', 'regional', '$', 'south']

[54] As regional Australia reels from several women’s deaths, advocates seek both policing and prevention [2024-04-26T15:00:12Z]
	- Counts:	 ['violence', 'woman', 'police', 'domestic', 'service', 'family', 'death', 'man', 'nsw', 'need']
	- TFIDF:	 ['violence', 'women', 'police', 'domestic', 'says', 'men', 'phillips', 'prevention', 'ticehurst', 'family']
	- LDA:		 ['government', 'people', 'state', 'make', ' ', 'community', 'need', 'regional', '$', 'south']

[100] Why have we been excluded? The students left out of Labor’s 

### Topic modelling with Non-negative Matrix Factorisation (NMF)


[NMF](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is a different algorithm for obtaining *topics* (a list of terms) from a document-term matrix. It also factorises the document-term matrix into 2 factor matrices: document-topic and topic-term.

In [28]:
# Set the number of topics
num_topics = 20

# Create the model
nmf_model = NMF(n_components=num_topics,init='random',beta_loss='frobenius')

# Fit the model to the data and use it to transform the data
doc_topic_nmf = nmf_model.fit_transform(tfidf_dt_matrix)

topic_term_nmf = nmf_model.components_

# THIS IS WHAT I WANT TO OPTIMISE HERE, AN APPROPRIATE CLUSTERING SHOWHING FUNDING

In [29]:
# Get the topics and their terms
nmf_topic_dict = {}
for index, topic in enumerate(topic_term_nmf):
    zipped = zip(feature_names, topic)
    top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
    #print(top_terms)
    top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
    nmf_topic_dict[f"topic_{index}"] = top_terms_list

# Print the topics with their terms    
for k,v in nmf_topic_dict.items():
    print(k)
    print(v)
    print()



topic_0
{'electric': 2.3234, 'rescind': 0.3, 'effective': 0.277, 'hughes': 0.2747, 'cornerstone': 0.2564, '1.5bn': 0.2305, 'treasurer': 0.2271, 'hydrology': 0.208, 'dissent': 0.1944, 'middleman': 0.1897}

topic_1
{'journey': 0.5658, 'stoke': 0.4158, 'total': 0.3325, 'furious': 0.3167, 'agree': 0.2442, 'layer': 0.2341, 'culinary': 0.2057, 'photo': 0.1987, 'josie': 0.1845, 'aid': 0.1822}

topic_2
{'appropriately': 2.883, 'appear': 1.018, 'guard': 0.8307, 'yougov': 0.6185, 'observe': 0.4534, 'tesla': 0.4509, 'robert': 0.4285, 'crush': 0.3972, 'excess': 0.3587, 'statewide': 0.3532}

topic_3
{'hughes': 1.4102, 'forensic': 1.3551, 'intake': 1.1367, 'coordinate': 0.8661, 'hugh': 0.7806, 'white': 0.3928, '70': 0.3759, 'agriculture': 0.3667, 'tara': 0.3617, 'concerned': 0.3611}

topic_4
{'warn': 1.4152, 'midwife': 1.1758, 'affairs': 0.619, 'affect': 0.614, 'veteran': 0.6016, 'warmth': 0.5345, 'punishment': 0.4744, 'mountain': 0.4668, 'limited': 0.448, 'assumption': 0.4477}

topic_5
{'77': 2.907

In [30]:
 list(top_topic.keys())


['government',
 'people',
 'state',
 'make',
 ' ',
 'community',
 'need',
 'regional',
 '$',
 'south']

#### Update the terms matrix

In [31]:
for idx,topic in enumerate(doc_topic_nmf):
    topic_num = topic.argmax()
    top_topic = lda_topic_dict[f"topic_{topic_num}"]
    terms_df['nmf'].iloc[idx] = list(top_topic.keys())

terms_df

C:\Users\Morgan\AppData\Local\Temp\ipykernel_18348\2376733025.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  terms_df['nmf'].iloc[idx] = list(top_topic.keys())


,count,tfidf,lda,nmf
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],"[drought, rain, country, state, network, farme...","[drought, rains, sheppard, channel, monaghan, ...","[government, people, state, make, , community...","[adani, carmichael, advantaged, rank, coordina..."
Queensland rejects Glencore carbon capture and storage proposal for Great Artesian Basin [2024-05-24T06:12:49Z],"[project, ccs, basin, artesian, great, glencor...","[ccs, artesian, glencore, basin, project, co2,...","[government, people, state, make, , community...","[detainee, 73, recount, bonza, scroll, cancell..."
Antarctic blast brings damaging winds and alpine blizzards to Victoria and NSW and possible snow in Queensland [2024-07-15T06:06:30Z],"[snow, south, weather, bring, condition, wind,...","[snow, belt, altitude, weather, metres, winds,...","[government, people, state, make, , community...","[drone, dairy, milk, heritage, world, rex, lis..."
"300,000ha Queensland cattle station bought for conservation after $21m donation [2024-04-16T23:00:03Z]","[government, protect, national, park, area, pr...","[000ha, park, linard, nature, opal, property, ...","[government, people, state, make, , community...","[detainee, 73, recount, bonza, scroll, cancell..."
Contrasting polls spell out disaster territory and green shoots for Labor ahead of Queensland election [2024-09-14T20:00:39Z],"[labor, party, seat, state, brisbane, election...","[labor, seats, redbridge, party, election, bri...","[government, people, state, make, , community...","[land, cattle, industry, emission, report, dat..."
...,...,...,...,...
Coalition won’t budge on free rapid Covid tests for all but concession card-holders are covered [2022-01-05T09:13:17Z],"[test, free, rapid, morrison, testing, governm...","[tests, nbsp, rapid, testing, morrison, antige...","[government, people, state, make, , community...","[court, mental, young, diversion, health, magi..."
"Over-12s should be able to get Covid vaccine in Australia without parental consent, say experts [2022-01-24T03:48:47Z]","[child, parent, consent, medical, vaccinate, v...","[consent, vaccine, parents, mature, medical, p...","[government, people, state, make, , community...","[doomadgee, thunderstorm, strongly, nicholson,..."
What do I do if I test positive for Covid? Here’s how to prepare for the result [2022-01-04T00:46:17Z],"[covid, health, positive, home, isolation, iso...","[covid, isolation, isolate, positive, health, ...","[government, people, state, make, , community...","[doomadgee, thunderstorm, strongly, nicholson,..."
NSW government to create a new reconstruction authority in response to flood report [2022-08-16T17:30:13Z],"[flood, agency, state, nsw, government, perrot...","[perrottet, agency, flood, resilience, disaste...","[government, people, state, make, , community...","[drone, dairy, milk, heritage, world, rex, lis..."


### Compare approaches

In [32]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print("\t- LDA:\t\t",doc['lda'])
    print("\t- NMF:\t\t",doc['nmf'])
    print()

[20] ‘Stuck in recovery mode’: far north Queensland still waiting to rebuild after ex-Cyclone Jasper  [2024-02-22T14:00:24Z]
	- Counts:	 ['community', 'flood', 'wujal', ' ', 'cyclone', 'home', 'make', 'insurance', 'ex', 'claim']
	- TFIDF:	 ['wujal', 'cyclone', 'entsch', 'ex', 'insurance', 'jasper', 'repairs', 'homes', 'mahon', 'community']
	- LDA:		 ['government', 'people', 'state', 'make', ' ', 'community', 'need', 'regional', '$', 'south']
	- NMF:		 ['government', 'people', 'state', 'make', ' ', 'community', 'need', 'regional', '$', 'south']

[82] Queensland introduces Australian-first law to allow midwives and nurses to prescribe abortion pills  [2023-11-30T03:02:56Z]
	- Counts:	 ['access', 'termination', 'midwife', 'pregnancy', 'abortion', 'step', 'woman', 'hospital', 'allow', 'introduce']
	- TFIDF:	 ['pregnancy', 'termination', 'midwives', 'access', 'abortion', 'nurses', 'ms', 'step', 'kelleher', 'terminations']
	- LDA:		 ['government', 'people', 'state', 'make', ' ', 'community',

Sometimes different approaches are used. Sometimes LDA is better, sometimes nmf. LDA/nmf use different. How do we evaluate which is best?

Andrew recommends not just using counts, but it can do pretty good sometimes

In [33]:
print(topic.argmax())


19


In [34]:
# ATTEMPT CLUSTERING
#https://youtu.be/i74DVqMsRWY?list=PL2VXyKi-KpYttggRATQVmgFcQst3z6OlX

km = KMeans(n_clusters=10, random_state=0, n_init="auto")
#km.fit(terms_df.reset_index()["nmf"])
#Fit model
km.fit(doc_topic_nmf)
# At this point it is still on the 10k wide matrix of frequencies, so need to displ

order_centroids = km.cluster_centers_.argsort()[:,::-1]
terms = nmf_model##.get_feature_names_out()

In [35]:
order_centroids

array([[16, 19,  1, 10,  7,  4,  8, 17,  9, 15,  0,  6,  3, 11,  5,  2,
        13, 18, 12, 14],
       [ 9, 17,  4,  1, 19, 16,  5,  2,  6,  7,  0, 15,  3, 10,  8, 13,
        12, 11, 18, 14],
       [11, 12,  7,  3, 15, 17,  9,  4, 16,  1, 19, 10, 13, 14,  0,  2,
        18,  6,  8,  5],
       [ 1,  8, 15,  9, 19,  4, 10,  0, 16, 11, 14,  5, 18,  7,  2, 13,
        17,  3, 12,  6],
       [19,  7,  3, 15, 16, 10,  1,  0,  4, 12, 14,  6,  9,  5, 13, 11,
        18,  8,  2, 17],
       [18,  8, 15,  9,  6, 11, 14,  7, 16,  3, 19,  1,  5,  4, 13, 12,
        10, 17,  2,  0],
       [14, 15,  3, 13, 10, 19,  4, 11,  0, 12,  7,  2,  8,  6, 16,  5,
         1, 17,  9, 18],
       [10, 15, 17, 13,  3,  9,  6,  5,  7, 16,  1,  0,  8, 18, 11,  4,
        12,  2, 14, 19],
       [15,  9,  6,  3,  4, 13, 16,  1, 10, 14,  7, 11, 17, 12,  8, 19,
        18,  5,  2,  0],
       [13, 15, 14,  6,  7,  1, 12,  3, 19, 16,  2, 17,  9,  4, 11,  0,
         8,  5, 18, 10]], dtype=int64)

In [36]:
#terms_df['nmf_factor'] = pd.factorize(terms_df[['nmf']])[0]